<a href="https://colab.research.google.com/github/Natural-State/agol-data-workflows/blob/master/code/Colab%20notebooks/20_GEDI_L2A_L2B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Extract canopy metrics measure from GEDI Level 2A and 2B platform

## Import gee and authenticate

In [4]:
import ee

In [5]:
# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=eH4X_1wVt02Gp7FKR0h_vERoD0TogWy29rxaIE2Fm24&tc=Pej9bzYo-jGZD-vY3uxqnDpqpU0KU3Dbwl9l4G2iyF0&cc=ERGZBbX4KZTipUDhNtn3BeKSx_0srskdNKg9u7NzwuE

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1AbUR2VOwAD7E_c3PoMht1jVgREbDebjgzNVF3JeWkNLj2DTiSq_odK7aejQ

Successfully saved authorization token.


## Input arguments for data extraction

In [50]:
# Area of interest
aoi = ee.FeatureCollection("projects/ns-agol-rs-data/assets/MKR_PACE")
aoi_name = "MKR_PACE"


# GEE layer ID - Level 2A
layer_name = "RS_063" # rh100

# GEE layer ID - Level 2B
layer_dict = {
    "cover": "RS_064",
    "pai": "RS_065",
    "pavd_z25": "RS_066"
}



## GEDI L2A



### Process

In [20]:
gedi2A = ee.ImageCollection("LARSE/GEDI/GEDI02_A_002_MONTHLY")

def qualityMask(image):
    return image.updateMask(image.select('quality_flag').eq(1)).updateMask(image.select('degrade_flag').eq(0))

canopy_height = gedi2A \
             .map(qualityMask) \
             .select("rh100") \
             .mean() \
             .clip(aoi)

print(gedi2A.size().getInfo())
print(canopy_height.getInfo())

47650
{'type': 'Image', 'bands': [{'id': 'rh100', 'data_type': {'type': 'PixelType', 'precision': 'double'}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}]}


### Export

In [22]:
task1 = ee.batch.Export.image.toDrive(image = canopy_height,
                                     region = aoi.geometry(),
                                     description = "EXPORT IMAGE TO DRIVE",
                                     folder = "GEE_exports",
                                     fileNamePrefix = layer_name + "_" + aoi_name,
                                     scale = 25,
                                     maxPixels = 10e12
                                     )
task1.start()

In [32]:
task_id = task1.status()['id']
task_state = task1.status()['state']
print(task_id, task_state)

DDMZTRWM4S4CII4PRICOPKY2 COMPLETED


## GEDI L2B

### Process

In [46]:
gedi2B = ee.ImageCollection('LARSE/GEDI/GEDI02_B_002_MONTHLY')

def qualityMask(image):
    return image.updateMask(image.select('l2b_quality_flag').eq(1)).updateMask(image.select('degrade_flag').eq(0))

def gedi_image(metric):
    return gedi2B \
         .map(qualityMask) \
         .select(metric) \
         .mean() \
         .clip(aoi) \
         .set({"canopy_metric": metric})

cover = gedi_image("cover")
pai = gedi_image("pai")
pavd_z25 = gedi_image("pavd_z25")

print(cover.getInfo())
print(cover.get("canopy_metric").getInfo())

print(pavd_z25.getInfo())
print(pavd_z25.get("canopy_metric").getInfo())

{'type': 'Image', 'bands': [{'id': 'cover', 'data_type': {'type': 'PixelType', 'precision': 'double'}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}], 'properties': {'canopy_metric': 'cover'}}
cover
{'type': 'Image', 'bands': [{'id': 'pavd_z25', 'data_type': {'type': 'PixelType', 'precision': 'double'}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}], 'properties': {'canopy_metric': 'pavd_z25'}}
pavd_z25


### Export

In [33]:
output_name = f"{layer_dict[cover.get('canopy_metric').getInfo()]}_{aoi_name}"
print(output_name)

task2 = ee.batch.Export.image.toDrive(image = cover,
                                     region = aoi.geometry(),
                                     description = "EXPORT IMAGE TO DRIVE",
                                     folder = "GEE_exports",
                                     fileNamePrefix = output_name,
                                     scale = 25,
                                     maxPixels = 10e12
                                     )
task2.start()

RS_064_MKR_PACE


In [34]:
output_name = f"{layer_dict[pai.get('canopy_metric').getInfo()]}_{aoi_name}"
print(output_name)

task3 = ee.batch.Export.image.toDrive(image = pai,
                                     region = aoi.geometry(),
                                     description = "EXPORT IMAGE TO DRIVE",
                                     folder = "GEE_exports",
                                     fileNamePrefix = output_name,
                                     scale = 25,
                                     maxPixels = 10e12
                                     )
task3.start()

RS_065_MKR_PACE


In [51]:
output_name = f"{layer_dict[pavd_z25.get('canopy_metric').getInfo()]}_{aoi_name}"
print(output_name)

task4 = ee.batch.Export.image.toDrive(image = pavd_z25,
                                     region = aoi.geometry(),
                                     description = "EXPORT IMAGE TO DRIVE",
                                     folder = "GEE_exports",
                                     fileNamePrefix = output_name,
                                     scale = 25,
                                     maxPixels = 10e12
                                     )
task4.start()

RS_066_MKR_PACE
